In [35]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
# from extract_graph import dic_to_sparse
from util import get_path, shift_skeleton
from plotutil import show_im,overlap, show_im_rgb, plot_nodes, plot_nodes_from_list,plot_t_tp1
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton,clean
import networkx as nx
from node_id import second_identification, whole_movement_identification,first_identification,relabel_nodes, clean_nodes, orient
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list
from sparse_util import dilate, zhangSuen
from realign import realign, reconnect
from util import get_path
import pandas as pd
from datetime import datetime,timedelta
import ast

In [11]:
plate = 13
date_init = datetime(2020,7,1,19,57)

In [12]:
dates_datetime = [date_init+timedelta(hours=4)*i for i in range(5)]

In [13]:
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime]

In [ ]:
dates.remove('0705_1957')

In [15]:
mat_skels=[read_mat(get_path(date,plate,True))['skel'] for date in dates]

In [16]:
dic_skels=[dic_to_sparse(mat_skel) for mat_skel in mat_skels]

In [17]:
skeleton_docs=[sparse_to_doc(sparse_skel) for sparse_skel in dic_skels]

In [37]:
nx_graph_poss=[]
for date in dates:
    nx_graph_poss.append(generate_nx_graph(pd.read_csv(get_path(date,plate,True,extension='_raw_aligned_skeleton.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}),labeled=True))

In [38]:
nx_graph_poss=[]
for date in dates:
    nx_graph_poss.append(generate_nx_graph(pd.read_csv(get_path(date,plate,True,extension='_raw_aligned_skeleton.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}),labeled=True))
# nx_graph_poss=[generate_nx_graph(from_sparse_to_graph(realign(skeleton_doc,skeleton_docs[0],2))) for skeleton_doc in skeleton_docs]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [20]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-1
downstream_graphs=[nx_graph_pruned[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,-1,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(nx_graph_pruned[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_posscurrent_node

i= 3
number removed first phase 77
number removed second phase 447
2 0 [8802 6718]
2 1 [8808 6734]
i= 2
number removed first phase 106
number removed second phase 349
2 0 [8813 6740]
2 1 [8802 6718]
2 2 [8808 6734]
i= 1
number removed first phase 152
number removed second phase 595
3 0 [9658 6714]
3 1 [9662 6724]
3 2 [9658 6713]
3 3 [9659 6719]
i= 0
number removed first phase 176
number removed second phase 580
2 0 [9661 6729]
2 1 [9658 6714]
2 2 [9662 6724]
2 3 [9658 6713]
2 4 [9659 6719]


In [28]:
nx_graph_pruned=downstream_graphs
poss_aligned=downstream_poss

In [11]:
# for i, date in enumerate(dates):
#     sparse.save_npz(f'Data/skeleton_{date}_{plate}_full_clean',sparse.csc_matrix(skeleton_docs_cleaned[i]))

In [14]:
for i, date in enumerate(dates):
    from_nx_to_tab(nx_graph_pruned[i],poss_aligned[i]).to_csv(get_path(date,plate,True,extension='_full_labeled_matlab.csv'))
    from_nx_to_tab_matlab(nx_graph_pruned[i],poss_aligned[i]).to_csv(get_path(date,plate,True,extension='_full_labeled.csv'))
#     tabs[i].to_csv(get_path(date,plate,True,extension='_full.csv'))
    

In [30]:
for i, date in enumerate(dates):
    from_nx_to_tab(nx_graph_pruned[i],poss_aligned[i]).to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
    from_nx_to_tab_matlab(nx_graph_pruned[i],poss_aligned[i]).to_csv(f'Data/graph_{date}_{plate}_full_labeled_matlab.csv')
# for i, date in enumerate(dates):
#     tabs_labeled[i].to_csv(f'Data/graph_{date}_{plate}_full_labeled.csv')
#     tabs[i].to_csv(f'Data/graph_{date}_{plate}_full.csv')

In [29]:
connections_growth_pattern = [whole_movement_identification(nx_graph_pruned[i],nx_graph_pruned[i+1],poss_aligned[i],poss_aligned[i+1]) for i in range(len(dates)-1)]
tip_origins = [c[0] for c in connections_growth_pattern]
growth_pattern = [c[1] for c in connections_growth_pattern]
for i in range(len(tip_origins)):
    connections_pixel_list_to_tab(tip_origins[i],growth_pattern[i]).to_csv(get_path(dates[i],plate,True,extension='_connection.csv'))

0.0
0.13297872340425532
0.26595744680851063
0.39893617021276595
0.5319148936170213
0.6648936170212766
0.7978723404255319
0.9308510638297872
0.0
0.12531328320802004
0.2506265664160401
0.37593984962406013
0.5012531328320802
0.6265664160401002
0.7518796992481203
0.8771929824561403
0.0
0.11547344110854503
0.23094688221709006
0.3464203233256351
0.4618937644341801
0.5773672055427251
0.6928406466512702
0.8083140877598153
0.9237875288683602
0.0
0.10822510822510822
0.21645021645021645
0.3246753246753247
0.4329004329004329
0.5411255411255411
0.6493506493506493
0.7575757575757576
0.8658008658008658
0.974025974025974


In [31]:
def connections_pixel_list_to_tab(origin_tips,pattern_growth):
    column_names = ['tip_origin',"nodes_from_tip", "growth_pattern"]
    tab = pd.DataFrame(columns=column_names)
    pattern_growth={tip : [[(pixel[0],pixel[1]) for pixel in branch] for branch in pattern_growth[tip]] for tip in pattern_growth.keys()}
    for tip in origin_tips.keys():
        new_line=pd.DataFrame({"tip_origin" : [tip],"nodes_from_tip" :[origin_tips[tip]], "growth_pattern" : [pattern_growth[tip]]})
        tab=tab.append(new_line,ignore_index=True)
    return(tab)
# for i in range(len(tip_origins)):
#     connections_pixel_list_to_tab(tip_origins[i],growth_pattern[i]).to_csv(get_path(dates[i],plate,True,extension='_connection.csv'))
for i in range(len(tip_origins)):
    connections_pixel_list_to_tab(tip_origins[i],growth_pattern[i]).to_csv(f'Data/graph_{dates[i]}_{plate}_connection.csv')

In [ ]:
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
nx_graph_pruned=[prune_graph(nx_graph) for nx_graph in nx_graphs_aligned]

In [ ]:
xbegin=14000
xend = 18000
ybegin = 8000
yend=16000
sub_nx_graphs = [sub_graph(nx_graph,poss_aligned[i],xbegin,xend,ybegin,yend) for i, nx_graph in enumerate(nx_graph_pruned)]
# sub_skeleton= [generate_skeleton(nx_graph,(xend-xbegin,yend-ybegin),(xbegin,ybegin)) for nx_graph in sub_nx_graphs]

In [13]:
downstream_graphs = []
downstream_pos = []
begin=len(dates)-2
downstream_graphs=[sub_nx_graphs[begin]]
downstream_poss=[poss_aligned[begin]]
for i in range (begin-1,0,-1):
    print("i=",i)
    new_graphs,new_poss = second_identification(sub_nx_graphs[i],downstream_graphs[0],poss_aligned[i],downstream_poss[0],50,downstream_graphs[1:],downstream_poss[1:],tolerance=30)
    downstream_graphs=new_graphs
    downstream_poss=new_poss

i= 3
number removed first phase 19
number removed second phase 36
116 0 [16887  8355]
116 1 [16893  8366]
i= 2
number removed first phase 13
number removed second phase 27
121 0 [17317  9088]
121 1 [17312  9091]
121 2 [17311  9107]
i= 1
number removed first phase 12
number removed second phase 32
119 0 [17318  9090]
119 1 [17317  9088]
119 2 [17312  9091]
119 3 [17311  9107]


In [65]:
degree3_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)>=3] for nx_graph in downstream_graphs]

In [69]:
t=0
tp1=1
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],downstream_poss[t],downstream_poss[tp1],sub_skeleton[t+1].todense(),sub_skeleton[tp1+1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
skeleton_docs_cleaned= [generate_skeleton(nx_graph,skeleton_docs_aligned[i].shape) for i,nx_graph in enumerate(nx_graph_pruned)]